## Renaming to hash index

In [7]:
json_files = [
    'summaries/foundational_mathematics_youtube_videos_with_transcripts_and_summaries.json',
    'summaries/classical_machine_learning_youtube_videos_with_transcripts_and_summaries.json',
    'summaries/programming_fundamentals_youtube_videos_with_transcripts_and_summaries.json',
    'summaries/mlops_youtube_videos_with_transcripts_and_summaries.json',
    'summaries/deep_learning_youtube_videos_with_transcripts_and_summaries.json',
]

audio_folders = [
    "compressed_outputs/foundational_mathematics_youtube_videos",
    "compressed_outputs/classical_machine_learning_youtube_videos",
    "compressed_outputs/programming_fundamentals_youtube_videos",
    "compressed_outputs/mlops_youtube_videos",
    "compressed_outputs/deep_learning_youtube_videos",
]

In [8]:
import json
from tqdm import tqdm
import pandas as pd
import hashlib

In [9]:
dataframes = []

for file in json_files:
    with open(file, 'r') as f:
        df = pd.DataFrame([json.loads(line) for line in f])
        dataframes.append(df)


for df, folder in zip(dataframes, audio_folders):
    for idx, row in df.iterrows():
        url = row["URL"]
        hash_key = hashlib.md5(url.encode()).hexdigest()
        
        old_idx = idx + 1
        old_path = os.path.join(folder, f"summary_{old_idx}.mp3")
        new_path = os.path.join(folder, f"summary_{hash_key}.mp3")

        if os.path.exists(old_path):
            os.rename(old_path, new_path)
            # print(f"Renamed: {old_path} → {new_path}")
        else:
            print(f"Missing file: {old_path}")

## Compressing

In [1]:
import os
from pydub import AudioSegment
from tqdm import tqdm

input_root = "audio_outputs"
output_root = "compressed_outputs"

In [2]:
target_bitrate = "512k"
target_sample_rate = 48000
target_channels = 4

In [3]:
for root, dirs, files in os.walk(input_root):
    for file in tqdm(files):
        if file.lower().endswith((".mp3", ".wav", ".flac", ".ogg", ".m4a")):
            input_path = os.path.join(root, file)
            relative_path = os.path.relpath(input_path, input_root)
            output_path = os.path.join(output_root, os.path.splitext(relative_path)[0] + ".mp3")

            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            audio = AudioSegment.from_file(input_path)
            audio = audio.set_channels(target_channels)
            audio = audio.set_frame_rate(target_sample_rate)
            audio.export(output_path, format="mp3", bitrate=target_bitrate)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 204/204 [04:21<00:00,  1.28s/it]
